In [ ]:
import seml
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

3

from poisson_atac.utils import model_type_map

seml_collection = "atac_to_atac"

def load_seml(seml_database, model_type_map):
    results = seml.get_results(seml_database, to_data_frame=True,  fields=["config", "config_hash", "result", "batch_id"],)

    results.loc[results['config.data.batch'].apply(isinstance,args = [list]), 'config.data.batch'] = 'Neurips\nall batches'
    results.loc[results['config.data.batch'].isna(), 'config.data.batch'] = 'Satpathy\net al.'
    

    results["config.model.model_type"] = results["config.model.model_type"].map(model_type_map)
    return results

results = load_seml(seml_collection, model_type_map)

results['config.model.model_type'] = results['config.model.model_type'] + results['config.setup.model_params.use_observed_lib_size'].astype(str) + results['config.setup.model_params.n_layers_encoder'].astype(str)

model_order = [model_type_map['count'], model_type_map["peakvi"]]

model_order

scib_table = pd.concat(results['result.embedding'].values, axis =1).T.dropna(axis=1, how='all')
scib_table

batch_metrics = ['PCR_batch', 'ASW_label/batch', 'iLISI', 'graph_conn']
bio_metrics = ['NMI_cluster/label', 'ARI_cluster/label', 'ASW_label', 'isolated_label_F1', 'isolated_label_silhouette', 'cLISI','trajectory']

scib_table = scib_table.loc[:, batch_metrics+bio_metrics]

#min_max_scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scib_table = pd.DataFrame(scaler.fit_transform(scib_table), columns=scib_table.columns)
weight_batch=0.4

score_group_batch = scib_table.loc[:, batch_metrics].mean(axis=1)
score_group_bio = scib_table.loc[:, bio_metrics].mean(axis=1)
score_all = (weight_batch*score_group_batch + (1-weight_batch)*score_group_bio)

scib_table.insert(0, "Overall", score_all)

scib_table["Model"] = results['config.model.model_type'].values
scib_table["Dataset"] = results['config.data.dataset'].values
scib_table["n_latent"] = results['config.setup.model_params.n_latent'].values

scib_table

sns.barplot(data=scib_table, x="n_latent", y="Overall", hue="Model",)
plt.legend(loc="lower right")

sns.scatterplot(data=scib_table, x="n_latent", y="trajectory", hue="Model")
plt.ylim(0.9,1)

# Reconstruction metrics

sns.scatterplot(data=results, x="config.setup.model_params.n_latent", y="result.average_precision", hue="config.model.model_type")

sns.scatterplot(data=results, x="config.setup.model_params.n_latent", y="result.rmse", hue="config.model.model_type")

sns.scatterplot(data=results, x="config.setup.model_params.n_latent", y="result.bce", hue="config.model.model_type")